In [ ]:
#base libraries for filemanagement etc
import os
import glob
from tqdm import tqdm
import random
import tarfile
import warnings
warnings.filterwarnings('ignore')

#data handling and visualiation libaries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

#image aug libraries
from PIL import Image #using library pillow

#pytorch libraries
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader,Dataset,WeightedRandomSampler,random_split
from torch.optim import Adam
from torch.autograd import Variable

import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid

import torchmetrics
import torchinfo

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision.transforms.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
import torchvision
from torchvision import models, transforms
from torchvision.datasets.folder import make_dataset
from PIL import Image
import matplotlib.pyplot as plt
import time
import os
import copy
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

%matplotlib inline
plt.ion()   # interactive mode

In [ ]:
pip install torchsummary

In [ ]:
from torchsummary import summary

In [ ]:
%matplotlib inline

In [ ]:
# use cuda if it is available else use cpu, cuda can handle more data and does training faster
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
RANDOM_SEED = 115
torch.cuda.manual_seed(RANDOM_SEED)

In [ ]:
img_dims = 224
target_size = (img_dims, img_dims) # size of dataset images


In [ ]:
# Define the dataset class
class sg_food_dataset(torch.utils.data.dataset.Dataset):
    def __init__(self, root, class_id, transform=None):
        self.class_id = class_id
        self.root = root
        all_classes = sorted(entry.name for entry in os.scandir(root) if entry.is_dir())
        if not all_classes:
            raise FileNotFoundError(f"Couldn't find any class folder in {directory}.")
        self.classes = [all_classes[x] for x in class_id]
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}

        self.samples = make_dataset(self.root, self.class_to_idx, extensions=('jpg'))
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, target = self.samples[idx]
        with open(path, "rb") as f:
            sample = Image.open(f).convert('RGB')
        if self.transform is not None:
            sample = self.transform(sample)
        return sample, target

In [ ]:
# Data augmentation and normalization for training
data_transforms = {
    'train': transforms.Compose([
        # Define data preparation operations for training set here.
        # Tips: Use torchvision.transforms
        #       https://pytorch.org/vision/stable/transforms.html
        #       Normally this should at least contain resizing (Resize) and data format converting (ToTensor).
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),     
        transforms.RandomVerticalFlip(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1 , saturation = 0.1), #random brightness, contrast etc

        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # ImageNet prior
    ]),
    'val': transforms.Compose([
        # Define data preparation operations for testing/validation set here.
        transforms.Resize((256,256), antialias=True),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # ImageNet prior
    ]),
    'test': transforms.Compose([
        # Define data preparation operations for testing/validation set here.
        transforms.Resize((256,256), antialias=True),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # ImageNet prior
    ]),
}


In [ ]:
data_dir = '/kaggle/input/sg-food/sg_food' 
subfolder = {'train': 'train', 'val': 'val', 'test': 'test'}

# Define the dataset
selected_classes = [0, 2, 4, 7, 9]
n_classes = len(selected_classes)
image_datasets = {x: sg_food_dataset(root=os.path.join(data_dir, subfolder[x]),
                                     class_id=selected_classes,
                                     transform=data_transforms[x]) 
                  for x in ['train', 'val','test']}
class_names = image_datasets['train'].classes
print('selected classes:\n    id: {}\n    name: {}'.format(selected_classes, class_names))

# Define the dataloader
batch_size = 16
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'val','test']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Visualise dataset

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs[:4])

imshow(out, title=[class_names[x] for x in classes[:4]])

In [ ]:
train_loader = dataloaders['train']
val_loader = dataloaders['val']
test_loader = dataloaders['test']

In [ ]:
# Labels
def get_model_results(model, data_loader):
    loop = tqdm(data_loader, total=len(data_loader))

    y_preds = []
    y_trues = []
    
    # Assuming device is defined elsewhere in your code
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    with torch.inference_mode():
        model.eval()
        for X, y_true in loop:
            X = X.to(device)
            y_true = y_true.to(device)
            
            y_pred = model(X)
            _, y_pred_class = torch.max(y_pred, 1)

            y_preds.extend(y_pred_class.data.cpu().numpy())
            y_trues.extend(y_true.data.cpu().numpy())

    print(f"{len(data_loader.dataset)} images")
    return y_preds, y_trues
from sklearn.metrics import confusion_matrix,roc_auc_score,accuracy_score,f1_score

def print_classification_report(y_trues,y_preds,dataset_name=''):
    print(f'Classification Report: {dataset_name}')
    # Build confusion matrix
    cf_matrix = confusion_matrix(y_trues, y_preds)
    df_cm = pd.DataFrame(cf_matrix, index = [i for i in selected_classes],
                         columns = [i for i in selected_classes])
    plt.figure(figsize = (12,7))
    sns.heatmap(df_cm, annot=True,fmt='g')

    # Set the axis labels and title
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    
    print('Accuracy: ', accuracy_score(y_trues,y_preds))
    print('F1 Score: ', f1_score(y_trues,y_preds,average='macro'))
    
    
    plt.show()
    plt.savefig(f"{dataset_name} Confusion Matrix.jpg", dpi=600)
    

# Knowledge Distillation training

### Define Teacher Vision Transformer

In [ ]:
from transformers import ViTModel
from transformers.modeling_outputs import SequenceClassifierOutput
import torch.nn as nn
import torch.nn.functional as F

# Define the model
class ViTForImageClassification(nn.Module):
    def __init__(self, num_labels, other_argument=None):
        super(ViTForImageClassification, self).__init__()
        # Your model initialization code here
        # Example:
        self.vit = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.vit.config.hidden_size, num_labels)
        self.num_labels = num_labels

    def forward(self, pixel_values, labels=None):
        # Your forward pass code here
        # Example:
        outputs = self.vit(pixel_values=pixel_values)
        output = self.dropout(outputs.last_hidden_state[:, 0])
        logits = self.classifier(output)

        if labels is not None:
            # Calculate loss
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return loss
        else:
            return logits

# initialize teacher and student

In [ ]:
# Instantiate the Vision Transformer model as the teacher model
teacher_type = "VisionTransformer"
teacher_model = ViTForImageClassification(5) # 5 classes

teacher_model = torch.load('/kaggle/input/4414visiontransformer/model_ViT.pt')
teacher_model.classifier = nn.Linear(in_features=768, out_features=5, bias=True)
teacher_model = teacher_model.to(device)
teacher_model.eval()
teacher_model

In [ ]:
# Instantiate the student model
student_type = "Resnet18"
student_model = models.resnet18(pretrained=True)

# change the final layer of ResNet Model for Transfer Learning
fc_inputs = student_model.fc.in_features
student_model.fc = nn.Sequential(nn.Linear(fc_inputs, 5)) # change no. of classes
student_model = student_model.to(device)
summary(student_model, input_size = (3,224,224))

In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

# training prep

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

def validation_step(model, batch):
    images, labels = batch 
    images, labels = images.to(device), labels.to(device)
    teacher_pred = teacher_model(images)                  # Generate predictions
    out = model(images) 
    loss = 0.5*(distillation_loss(out, teacher_pred, temperature)) + 0.5*(criterion(out, labels))   # Calculate loss
    acc = accuracy(out, labels)           # Calculate accuracy
    return {'val_loss': loss.detach(), 'val_acc': acc}

def validation_epoch_end(outputs):
    batch_losses = [x['val_loss'] for x in outputs]
    epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
    batch_accs = [x['val_acc'] for x in outputs]
    epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
    return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

def epoch_end(epoch, result):
    print("Epoch [{}], last_lr: {:.8f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
        epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc']))
    if result['val_acc']>0.93:
        torch.save(student_model.state_dict(), f'{student_type}-student_of_{teacher_type}_{temperature}_acc93.pth')

In [ ]:
# Define the distillation loss function
def distillation_loss(student_outputs, teacher_outputs, temperature):
    soft_teacher_outputs = F.softmax(teacher_outputs / temperature, dim=1)
    soft_student_outputs = F.log_softmax(student_outputs / temperature, dim=1)
    return F.kl_div(soft_student_outputs, soft_teacher_outputs, reduction='batchmean')


In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [validation_step(model,batch) for batch in val_loader]
    return validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr,temperature, teacher_model, student_model, train_loader, val_loader, 
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD, criterion = nn.CrossEntropyLoss()):
    torch.cuda.empty_cache()
    history = []
    
    # Set up cutom optimizer with weight decay
    optimizer = opt_func(student_model.parameters(), max_lr)#, weight_decay=weight_decay)
                        # optimizer for student
    
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_loader))
    
    for epoch in tqdm(range(epochs)):
        # Training Phase 
        teacher_model.eval()
        student_model.train()
        
        cls_losses = []
        div_losses = []
        losses = []
        lrs = []
        for batch in train_loader:
            teacher_model.eval()
            images, labels = batch 
            images, labels = images.to(device), labels.to(device)
            
            teacher_pred = teacher_model(images)                  # Generate predictions
            cls_pred = student_model(images)                  # Generate predictions
            
            
#             # Gradient clipping
#             if grad_clip: 
#                 nn.utils.clip_grad_value_(student_model.parameters(), grad_clip)

            cls_loss = criterion(cls_pred, labels) # Calculate loss
            div_loss = distillation_loss(cls_pred, teacher_pred, temperature)
            
            
            cls_losses.append(cls_loss)
            div_losses.append(div_loss)
            
            loss = 0.5 * cls_loss + 0.5 * div_loss
            
            losses.append(loss)
            loss.backward()
            
            optimizer.step()
            optimizer.zero_grad()
            
            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()
        
        # Validation phase
        result = evaluate(student_model, val_loader)
        
        result['div_loss'] = torch.stack(div_losses).mean().item()
        result['cls_loss'] = torch.stack(cls_losses).mean().item()
        
        result['train_loss'] = torch.stack(losses).mean().item()
        result['lrs'] = lrs
        
        epoch_end(epoch, result)
        history.append(result)
    return history

# train

In [ ]:
epochs = 20
max_lr = 1e-4
grad_clip = 0
weight_decay = 5e-4
opt_func = torch.optim.Adam
criterion = nn.CrossEntropyLoss()

In [ ]:
%%time
temperature = 3 # Temperature parameter for distillation
history = fit_one_cycle(epochs, max_lr,temperature, teacher_model,student_model, train_loader, val_loader, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)

In [ ]:
torch.save(student_model.state_dict(), f'{student_type}-student_of_{teacher_type}_{temperature}_9125.pth')

In [ ]:
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');
    
def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');
    
def plot_lrs(history):
    lrs = np.concatenate([x.get('lrs', []) for x in history])
    plt.plot(lrs)
    plt.xlabel('Batch no.')
    plt.ylabel('Learning rate')
    plt.title('Learning Rate vs. Batch no.');

In [ ]:
plot_accuracies(history)

In [ ]:
plot_losses(history)

In [ ]:
plot_lrs(history)

In [ ]:
def plot_losses(history):
    cls_losses = [x.get('cls_loss') for x in history]
    div_losses = [x.get('div_loss') for x in history]
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, 'k')
    plt.plot(val_losses, 'm')
    
    plt.plot(cls_losses, '-bx')
    plt.plot(div_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation', 'Class Loss','Div Loss'])
    plt.title('Loss vs. No. of epochs')
    
plot_losses(history)

In [ ]:
%%time
t_preds, t_trues = get_model_results(student_model,train_loader)
print_classification_report(t_trues,t_preds,'Train Dataset')

In [ ]:
%%time
val_preds, val_trues = get_model_results(student_model,val_loader)
print_classification_report(val_trues,val_preds,'Validation Dataset')

In [ ]:
%%time
test_preds, test_trues = get_model_results(student_model,test_loader)
print_classification_report(test_trues,test_preds,'Test Dataset')

# Test classifier with 93.12% val acc during training

In [ ]:
# Instantiate the student model
student_type = "Resnet18"
student_model1 = models.resnet18(pretrained=True)

# change the final layer of ResNet Model for Transfer Learning
fc_inputs = student_model1.fc.in_features
student_model1.fc = nn.Sequential(nn.Linear(fc_inputs, 5)) # change no. of classes

student_model1.load_state_dict(torch.load('/kaggle/working/Resnet18-student_of_VisionTransformer_3_93-12percent.pth'))
student_model1 = student_model1.to(device)
student_model1.eval()
summary(student_model1, input_size = (3,224,224))

In [ ]:
%%time
t_preds, t_trues = get_model_results(student_model1,train_loader)
print_classification_report(t_trues,t_preds,'Train Dataset')

In [ ]:
%%time
val_preds, val_trues = get_model_results(student_model1,val_loader)
print_classification_report(val_trues,val_preds,'Validation Dataset')

In [ ]:
%%time
test_preds, test_trues = get_model_results(student_model1,test_loader)
print_classification_report(test_trues,test_preds,'Test Dataset')